# Common Voice Separation Challenge

This is a classifying challenge built with a Bayes Classifier implementation.

- We have two `csv` files, one with data to train and other with data to validate;
- Both belong to the same dataset which holds 39 features and 5 classes;
- We use a self implementation of a bayesian classifier along with a cross validation strategy;
- The best classifier caught from CV is, then, used to classify data from the validation file;
- The result is stored on a response file whose content is to be verified;
- Wish me luck!

Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold

from BayesClassifier import BayesClassifier

### Import Data

In [2]:
dsTrain = pd.read_csv('./file/treino.csv')
features = dsTrain.columns[1:-1]

idKey = 'id'
classKey = 'y'
classes = dsTrain[classKey].unique()

Y = dsTrain[classKey].values
X = StandardScaler().fit_transform(dsTrain[features].values)


#### Visualize data without normalization

In [3]:
dsTrain.describe()

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38
count,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,...,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000
mean,1749.500000,5.801373,0.129776,-0.080697,0.124290,-0.069568,0.020910,-0.048900,0.008685,-0.027437,...,-0.005714,0.001124,-0.008123,0.006108,-0.008370,0.003370,-0.004579,0.001335,-0.004414,-0.006183
std,1010.507298,0.553708,0.123980,0.093510,0.092898,0.089135,0.052297,0.053688,0.035701,0.030207,...,0.009593,0.009048,0.012625,0.016424,0.013657,0.012869,0.010866,0.013420,0.009063,0.005669
min,0.000000,1.711399,-0.327801,-0.476553,-0.098704,-0.287921,-0.247465,-0.207162,-0.182348,-0.190235,...,-0.048417,-0.042460,-0.056412,-0.031993,-0.051446,-0.027959,-0.047215,-0.026848,-0.037930,-0.031556
25%,874.750000,5.647714,0.046752,-0.141731,0.047857,-0.147098,-0.018919,-0.094705,-0.018231,-0.045746,...,-0.011652,-0.004524,-0.014107,-0.003971,-0.012464,-0.004658,-0.008465,-0.006812,-0.008247,-0.010106
50%,1749.500000,5.966449,0.135367,-0.079790,0.110533,-0.059910,0.019772,-0.044049,0.005706,-0.026727,...,-0.005303,0.001016,-0.005500,0.000957,-0.004434,-0.000157,-0.001810,-0.002524,-0.002603,-0.006293
75%,2624.250000,6.147465,0.219378,-0.015827,0.209089,0.004390,0.057833,-0.005593,0.037291,-0.009213,...,0.000485,0.006401,0.000358,0.008538,0.000600,0.006309,0.002531,0.003849,0.001484,-0.002480
max,3499.000000,6.727103,0.628729,0.211273,0.376119,0.128229,0.224193,0.106031,0.122694,0.091944,...,0.027770,0.036939,0.033304,0.059907,0.022399,0.047134,0.020612,0.053480,0.018757,0.017067


#### Visualize normalized data

In [4]:
aux = list(features)
aux.append(classKey)
dsTrainNorm = pd.DataFrame(columns=aux)

dsTrainNorm[features] = StandardScaler().fit_transform(dsTrain[features].values)
dsTrainNorm[classKey] = dsTrain[classKey].values

dsTrainNorm.describe()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38
count,3.500000e+03,3.500000e+03,3.500000e+03,3.500000e+03,3.500000e+03,3.500000e+03,3.500000e+03,3.500000e+03,3.500000e+03,3.500000e+03,...,3.500000e+03,3.500000e+03,3.500000e+03,3.500000e+03,3.500000e+03,3.500000e+03,3.500000e+03,3.500000e+03,3.500000e+03,3.500000e+03
mean,7.085126e-16,-1.319579e-16,-1.207923e-16,-1.928616e-16,1.674851e-17,-3.552714e-17,1.157170e-16,-1.015061e-17,3.248195e-17,3.451208e-17,...,-7.511452e-17,-2.436147e-17,-3.425831e-17,3.349701e-17,-4.263256e-17,4.060244e-18,3.654220e-17,4.060244e-18,-4.770787e-17,-1.700227e-17
std,1.000143e+00,1.000143e+00,1.000143e+00,1.000143e+00,1.000143e+00,1.000143e+00,1.000143e+00,1.000143e+00,1.000143e+00,1.000143e+00,...,1.000143e+00,1.000143e+00,1.000143e+00,1.000143e+00,1.000143e+00,1.000143e+00,1.000143e+00,1.000143e+00,1.000143e+00,1.000143e+00
min,-7.387573e+00,-3.691261e+00,-4.233918e+00,-2.400759e+00,-2.450022e+00,-5.132497e+00,-2.948215e+00,-5.351605e+00,-5.390223e+00,-4.171927e+00,...,-4.452163e+00,-4.817661e+00,-3.825391e+00,-2.320175e+00,-3.154741e+00,-2.434869e+00,-3.924347e+00,-2.100408e+00,-3.698548e+00,-4.476027e+00
25%,-2.775491e-01,-6.697463e-01,-6.527989e-01,-8.228853e-01,-8.699246e-01,-7.617061e-01,-8.532867e-01,-7.540200e-01,-6.062083e-01,-6.382913e-01,...,-6.190686e-01,-6.242627e-01,-4.740195e-01,-6.137851e-01,-2.998673e-01,-6.239333e-01,-3.576205e-01,-6.071367e-01,-4.230086e-01,-6.919678e-01
50%,2.981701e-01,4.510473e-02,9.702076e-03,-1.481053e-01,1.083639e-01,-2.175288e-02,9.036800e-02,-8.343590e-02,2.350988e-02,5.091387e-02,...,4.284784e-02,-1.196470e-02,2.077990e-01,-3.137106e-01,2.881972e-01,-2.741291e-01,2.549013e-01,-2.875632e-01,1.998308e-01,-1.937498e-02
75%,6.251327e-01,7.228193e-01,6.938183e-01,9.129433e-01,8.298460e-01,7.061315e-01,8.067411e-01,8.013862e-01,6.034046e-01,6.682513e-01,...,6.462783e-01,5.833586e-01,6.718409e-01,1.479509e-01,6.569134e-01,2.283614e-01,6.544806e-01,1.873476e-01,6.508445e-01,6.533318e-01
max,1.672112e+00,4.025048e+00,3.122791e+00,2.711203e+00,2.219376e+00,3.887661e+00,2.886141e+00,3.193859e+00,3.952711e+00,5.372706e+00,...,3.491079e+00,3.958888e+00,3.281777e+00,3.276051e+00,2.253303e+00,3.401239e+00,2.318700e+00,3.886214e+00,2.556890e+00,4.101638e+00


### Train

In [7]:

cv = KFold(n_splits=2, shuffle=True, random_state=True)
cv.get_n_splits(X)

history = list()
# bestPca = None
maxAccuracy = -np.inf
bestClassifier = None

def getAccuracy(yHat: np.array, y: np.array) -> float:
    return sum(yHat == y) / y.shape[0]

i = 0
for trainIdx, testIdx in cv.split(X):
    i += 1

    xTest, yTest = X[testIdx], Y[testIdx]
    xTrain, yTrain = X[trainIdx], Y[trainIdx]

    # pca = PCA(.98).fit(xTrain)
    # xTrain = pca.transform(xTrain)
    # xTest = pca.transform(xTest)

    classifier = BayesClassifier().fit(xTrain, yTrain)
    yHat = classifier.predict(xTest)

    accuracy = getAccuracy(yHat, yTest)
    if accuracy > maxAccuracy:
        # bestPca = pca
        maxAccuracy = accuracy
        bestClassifier = classifier

    history.append({ 'accuracy': accuracy, 'classifier': classifier })

print(f'{i} folds tested')

2 folds tested


Check overall accuracy against source dataset

In [8]:
# xValidation = bestPca.transform(X)
xValidation = X
yHat = bestClassifier.predict(xValidation)
overallAccuracy = getAccuracy(yHat, Y)
print(f'overallAccuracy: {overallAccuracy}')

overallAccuracy: 0.8805714285714286


#### Write validation data

In [9]:
# Set data validation data
dsValidation = pd.read_csv('./file/validacao.csv')
xValidation = dsValidation[features].values

# Build result csv file
responseData = pd.DataFrame(columns=[idKey, classKey])
responseData[idKey] = dsValidation[idKey]
responseData[classKey] = bestClassifier.predict(xValidation)

responseData.to_csv('./sample.csv', index=False)